Mount google drive and setup project path for colab

In [16]:
import sys
import os

from google.colab import drive

GOOGLE_DRIVE_PATH = "/content/drive"
PROJECT_PATH = "/My Drive/Colab Notebooks/FinGPT/"

drive.mount(GOOGLE_DRIVE_PATH, force_remount=True)

sys.path.insert(0, f"{GOOGLE_DRIVE_PATH}{PROJECT_PATH}")
os.chdir(f"{GOOGLE_DRIVE_PATH}{PROJECT_PATH}")

Mounted at /content/drive


Install the latest version of the transformers library

In [2]:
!pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00


Save the facebook/mms-tts-fin model.

In [4]:
# download the model

from transformers import VitsModel, AutoTokenizer
model = VitsModel.from_pretrained("facebook/mms-tts-fin")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-fin")

model_save_path = "models/facebook_mms_tts_fin"
# Save the model and tokenizer to the specified directory
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

Some weights of the model checkpoint at facebook/mms-tts-fin were not used when initializing VitsModel: ['posterior_encoder.wavenet.res_skip_layers.2.weight_v', 'flow.flows.2.wavenet.res_skip_layers.3.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'posterior_encoder.wavenet.res_skip_layers.14.weight_v', 'flow.flows.1.wavenet.in_layers.2.weight_g', 'flow.flows.3.wavenet.in_layers.3.weight_v', 'posterior_encoder.wavenet.res_skip_layers.2.weight_g', 'posterior_encoder.wavenet.in_layers.7.weight_v', 'flow.flows.1.wavenet.res_skip_layers.2.weight_v', 'posterior_encoder.wavenet.res_skip_layers.12.weight_g', 'flow.flows.2.wavenet.in_layers.2.weight_g', 'posterior_encoder.wavenet.in_layers.0.weight_v', 'posterior_encoder.wavenet.in_layers.3.weight_g', 'posterior_encoder.wavenet.res_skip_layers.4.weight_v', 'posterior_encoder.wavenet.res_skip_layers.14.weight_g', 'posterior_encoder.wavenet.in_layers.0.weight_g', 'flow.flows.2.wavenet.res_skip_layers.1.weight_g', 'posterior_encoder.wave

('models/facebook_mms_tts_fin/tokenizer_config.json',
 'models/facebook_mms_tts_fin/special_tokens_map.json',
 'models/facebook_mms_tts_fin/vocab.json',
 'models/facebook_mms_tts_fin/added_tokens.json')

Load model from local directory and perform text To Speech model


In [49]:
# Loading the model and tokenizer
from transformers import VitsModel, AutoTokenizer
import torch
import scipy
from IPython.display import Audio

# Define the directory path where you saved the model
model_directory = "./models/facebook_mms_tts_fin/"

# Load the model and tokenizer from the saved directory
model = VitsModel.from_pretrained(model_directory)
tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Now, you can use the loaded model and tokenizer for inference or fine-tuning.
text = "Hei, hauska tavata. Sinnikäs sää tänään"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform

scipy.io.wavfile.write("techno.wav", rate=model.config.sampling_rate,
data=output.cpu().float().numpy().T)

Audio(output, rate=model.config.sampling_rate)

Save Speech Recognition Model

In [46]:
from transformers import AutoProcessor, AutoModelForCTC

# Load the processor and model
processor = AutoProcessor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-finnish")
model = AutoModelForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-finnish")

# Specify the directory path where you want to save the processor and model
save_directory = "./models/speech_recognition"

# Save the processor and model to the specified directory
processor.save_pretrained(save_directory)
model.save_pretrained(save_directory)


Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-finnish were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-finnish and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
Y

Load model from local directory and perform Speech Recognition

In [47]:
from transformers import AutoProcessor, AutoModelForCTC, AutoConfig
import soundfile as sf
import torch

from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Specify the directory path where you saved the processor and model
load_directory = "./models/speech_recognition/"  # Replace with the actual path to your saved models

# Load the processor and model from the saved directory
loaded_processor = Wav2Vec2Processor.from_pretrained(load_directory, sampling_rate=16000)
loaded_model = Wav2Vec2ForCTC.from_pretrained(load_directory)


# Specify the path to your audio file
audio_file_path = "techno.wav"  # Replace with the actual path to your audio file

# Load the audio file
audio_input, sample_rate = sf.read(audio_file_path)

# Convert the audio data to a PyTorch tensor
audio_input_tensor = torch.from_numpy(audio_input)

# Preprocess the audio data using the loaded processor
input_features = loaded_processor(audio_input_tensor, return_tensors="pt")

# Perform inference with the model
with torch.no_grad():
    output = loaded_model(**input_features)

# Decode the CTC output to get the final transcription
predicted_ids = torch.argmax(output.logits, dim=-1)
transcription = loaded_processor.batch_decode(predicted_ids)

# Print the transcription
print(transcription)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


['HEI HAUSKA TAVATA SINNIKÄS SÄÄ TÄNÄÄN']
